In [ ]:
import imutils
import cv2
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 8]

In [ ]:
vid = cv2.VideoCapture('/var/tmp/cam1.mp4')
skips = int(vid.get(cv2.CAP_PROP_FPS) / 2)
def make_frames():
    i = 0
    while True:
        status, frame = vid.read()
        if not status:
            break
        i += 1
        if i == skips:
            i = 0
            yield cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

frames = list(make_frames())

In [ ]:
len(frames)

In [ ]:
def show(*args):
    if len(args) > 1:
        imgs = np.hstack(tuple(args))
    else:
        imgs = args[0]
    plt.imshow(imgs)
    plt.show()
    
left = frames[39]
right = frames[40]

show(left, right)

In [ ]:
def pp(frame):
    frame = imutils.resize(frame, width = 1000)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    frame = cv2.GaussianBlur(frame, (21, 21), 0)
    return frame
diff = cv2.absdiff(pp(left), pp(right))
show(diff)

In [ ]:
_, thresh = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)
show(thresh)

In [ ]:
dilate = cv2.dilate(thresh, None, iterations = 2)
show(dilate)

In [ ]:
onto = left.copy()
scale = left.shape[0]/float(dilate.shape[0])
cnts, _ = cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for c in cnts:
    if cv2.contourArea(c) < 500:
        continue
    (x, y, w, h) = cv2.boundingRect(c)
    x = int(x*scale)
    y = int(y*scale)
    w = int(w*scale)
    h = int(h*scale)
    cv2.rectangle(onto, (x,y), (x+w,y+h), (0,255,0), 3)
#onto = cv2.drawContours(onto, cnts, -1, (0,255,0), 3)
show(onto)
